# ODR Fitting Test Notebook

This notebook tests the new ODR fitting functionality for fracture toughness data.

In [ ]:
import sys

sys.path.append("../src")

import numpy as np
import pandas as pd
from uncertainties import ufloat
from sft.odr_fit import fit_fracture_toughness_model, FractureToughnessResult
from sft.prepare import build_fracture_toughness_df

print("Imports successful")

Imports successful


## Create Test Data

Create synthetic fracture toughness data for testing.

In [ ]:
def create_test_data(n_points_per_series=20):
    """Create synthetic test data with known parameters."""

    # True parameters
    true_params = {
        "GIc_1": 0.5,
        "GIIc_1": 0.3,
        "GIc_2": 0.6,
        "GIIc_2": 0.4,
        "GIc_3": 0.7,
        "GIIc_3": 0.5,
        "n": 2.0,
        "m": 2.0,
    }

    data_rows = []

    for series_idx, series_name in enumerate(["1", "2", "3"]):
        GIc_true = true_params[f"GIc_{series_idx + 1}"]
        GIIc_true = true_params[f"GIIc_{series_idx + 1}"]
        n_true = true_params["n"]
        m_true = true_params["m"]

        # Generate Gi values
        gi_values = np.linspace(0.1 * GIc_true, 0.9 * GIc_true, n_points_per_series)

        # Calculate corresponding Gii values using interaction law
        # ((Gi/GIc)^(1/n) + (Gii/GIIc)^(1/m)) = 1
        # Therefore: Gii = GIIc * (1 - (Gi/GIc)^(1/n))^(1/(1/m))
        gii_values = GIIc_true * (1 - (gi_values / GIc_true) ** (1 / n_true)) ** (
            1 / (1 / m_true)
        )

        # Add some noise
        gi_noise = np.random.normal(0, 0.02 * GIc_true, n_points_per_series)
        gii_noise = np.random.normal(0, 0.02 * GIIc_true, n_points_per_series)

        gi_values += gi_noise
        gii_values += gii_noise

        # Ensure positive values
        gi_values = np.maximum(gi_values, 0.01 * GIc_true)
        gii_values = np.maximum(gii_values, 0.01 * GIIc_true)

        # Create uncertainties (5% of values)
        gi_uncertainties = 0.05 * gi_values
        gii_uncertainties = 0.05 * gii_values

        # Add data for both manual and video sources
        for source in ["manual", "video"]:
            for i in range(n_points_per_series):
                data_rows.append(
                    {
                        "source": source,
                        "series": series_name,
                        "GIc": ufloat(gi_values[i], gi_uncertainties[i]),
                        "GIIc": ufloat(gii_values[i], gii_uncertainties[i]),
                    }
                )

    df = pd.DataFrame(data_rows)
    df = df.set_index(["source", "series"]).sort_index()

    return df, true_params


# Set random seed for reproducible results
np.random.seed(42)

# Create test data
test_df, true_params = create_test_data(n_points_per_series=15)

print(f"Created test DataFrame with shape: {test_df.shape}")
print(f"Index levels: {test_df.index.names}")
print(f"Columns: {test_df.columns.tolist()}")
print(f"\nTrue parameters: {true_params}")

# Show sample of data
print("\nSample data:")
print(test_df.head())

Created test DataFrame with shape: (90, 2)
Index levels: ['source', 'series']
Columns: ['GIc', 'GIIc']

True parameters: {'GIc_1': 0.5, 'GIIc_1': 0.3, 'GIc_2': 0.6, 'GIIc_2': 0.4, 'GIc_3': 0.7, 'GIIc_3': 0.5, 'n': 2.0, 'm': 2.0}

Sample data:
                           GIc             GIIc
source series                                  
manual 1       0.0550+/-0.0027    0.137+/-0.007
       1         0.077+/-0.004    0.103+/-0.005
       1         0.114+/-0.006    0.088+/-0.004
       1         0.151+/-0.008  0.0634+/-0.0032
       1         0.162+/-0.008  0.0462+/-0.0023


## Test Basic Fitting

In [ ]:
# Test basic fitting with fixed n, m
print("Testing ODR fitting with fixed n=2.0, m=2.0...")

try:
    result = fit_fracture_toughness_model(test_df, n=2.0, m=2.0)

    print("\nFitting successful!")
    print(f"Reduced χ²: {result.reduced_chi_squared:.4f}")
    print(f"R²: {result.r_squared:.4f}")
    print(f"p-value: {result.p_value:.4f}")
    print(f"Degrees of freedom: {result.degrees_of_freedom}")

    print("\nFitted parameters vs true values:")
    print(
        f"GIc_1: {result.GIc_1:.4f} ± {result.GIc_1_err:.4f} (true: {true_params['GIc_1']:.3f})"
    )
    print(
        f"GIIc_1: {result.GIIc_1:.4f} ± {result.GIIc_1_err:.4f} (true: {true_params['GIIc_1']:.3f})"
    )
    print(
        f"GIc_2: {result.GIc_2:.4f} ± {result.GIc_2_err:.4f} (true: {true_params['GIc_2']:.3f})"
    )
    print(
        f"GIIc_2: {result.GIIc_2:.4f} ± {result.GIIc_2_err:.4f} (true: {true_params['GIIc_2']:.3f})"
    )
    print(
        f"GIc_3: {result.GIc_3:.4f} ± {result.GIc_3_err:.4f} (true: {true_params['GIc_3']:.3f})"
    )
    print(
        f"GIIc_3: {result.GIIc_3:.4f} ± {result.GIIc_3_err:.4f} (true: {true_params['GIIc_3']:.3f})"
    )
    print(f"n: {result.n:.4f} ± {result.n_err:.4f} (fixed at 2.0)")
    print(f"m: {result.m:.4f} ± {result.m_err:.4f} (fixed at 2.0)")

    print(f"\nConvergence info: {result.convergence_info}")

except Exception as e:
    print(f"Error during fitting: {e}")
    import traceback

    traceback.print_exc()

Testing ODR fitting with fixed n=2.0, m=2.0...
Error during fitting: an implicit model cannot use response data


Traceback (most recent call last):
  File "/var/folders/ds/xp1wt8vj20gc32mpnj0q1q_h0000gn/T/ipykernel_88505/2422198488.py", line 5, in <module>
    result = fit_fracture_toughness_model(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/m332023/Repos/publications/seasonal-fracture-toughness/notebooks/../src/sft/odr_fit.py", line 400, in fit_fracture_toughness_model
    odr_obj = odr.ODR(odr_data, odr_model, beta0=initial_params, ifixb=fix_list)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/m332023/.pyenv/versions/3.12.3/lib/python3.12/site-packages/scipy/odr/_odrpack.py", line 789, in __init__
    self._check()
  File "/Users/m332023/.pyenv/versions/3.12.3/lib/python3.12/site-packages/scipy/odr/_odrpack.py", line 801, in _check
    raise OdrError("an implicit model cannot use response data")
scipy.odr._odrpack.OdrError: an implicit model cannot use response data


## Test Fitting with Free Parameters

In [ ]:
# Test fitting with n, m as free parameters
print("Testing ODR fitting with free n, m parameters...")

try:
    result_free = fit_fracture_toughness_model(
        test_df,
        bounds={
            "GIc_min": 0.1,
            "GIc_max": 2.0,
            "GIIc_min": 0.1,
            "GIIc_max": 2.0,
            "n_min": 0.5,
            "n_max": 5.0,
            "m_min": 0.5,
            "m_max": 5.0,
        },
    )

    print("\nFitting successful!")
    print(f"Reduced χ²: {result_free.reduced_chi_squared:.4f}")
    print(f"R²: {result_free.r_squared:.4f}")
    print(f"p-value: {result_free.p_value:.4f}")
    print(f"Degrees of freedom: {result_free.degrees_of_freedom}")

    print("\nFitted parameters vs true values:")
    print(
        f"GIc_1: {result_free.GIc_1:.4f} ± {result_free.GIc_1_err:.4f} (true: {true_params['GIc_1']:.3f})"
    )
    print(
        f"GIIc_1: {result_free.GIIc_1:.4f} ± {result_free.GIIc_1_err:.4f} (true: {true_params['GIIc_1']:.3f})"
    )
    print(
        f"GIc_2: {result_free.GIc_2:.4f} ± {result_free.GIc_2_err:.4f} (true: {true_params['GIc_2']:.3f})"
    )
    print(
        f"GIIc_2: {result_free.GIIc_2:.4f} ± {result_free.GIIc_2_err:.4f} (true: {true_params['GIIc_2']:.3f})"
    )
    print(
        f"GIc_3: {result_free.GIc_3:.4f} ± {result_free.GIc_3_err:.4f} (true: {true_params['GIc_3']:.3f})"
    )
    print(
        f"GIIc_3: {result_free.GIIc_3:.4f} ± {result_free.GIIc_3_err:.4f} (true: {true_params['GIIc_3']:.3f})"
    )
    print(
        f"n: {result_free.n:.4f} ± {result_free.n_err:.4f} (true: {true_params['n']:.3f})"
    )
    print(
        f"m: {result_free.m:.4f} ± {result_free.m_err:.4f} (true: {true_params['m']:.3f})"
    )

except Exception as e:
    print(f"Error during fitting: {e}")
    import traceback

    traceback.print_exc()

## Test Grid Search

In [ ]:
# Test with grid search for better initialization
print("Testing ODR fitting with grid search...")

try:
    result_grid = fit_fracture_toughness_model(
        test_df,
        grid_search=True,
        grid_points=5,  # Small grid for testing
        bounds={
            "GIc_min": 0.1,
            "GIc_max": 2.0,
            "GIIc_min": 0.1,
            "GIIc_max": 2.0,
            "n_min": 1.0,
            "n_max": 3.0,
            "m_min": 1.0,
            "m_max": 3.0,
        },
    )

    print("\nGrid search fitting successful!")
    print(f"Reduced χ²: {result_grid.reduced_chi_squared:.4f}")
    print(f"R²: {result_grid.r_squared:.4f}")
    print(f"p-value: {result_grid.p_value:.4f}")

    print("\nFitted parameters vs true values:")
    print(
        f"n: {result_grid.n:.4f} ± {result_grid.n_err:.4f} (true: {true_params['n']:.3f})"
    )
    print(
        f"m: {result_grid.m:.4f} ± {result_grid.m_err:.4f} (true: {true_params['m']:.3f})"
    )

except Exception as e:
    print(f"Error during grid search fitting: {e}")
    import traceback

    traceback.print_exc()

## Compare Results

In [ ]:
# Compare all fitting approaches
print("\nComparison of fitting approaches:")
print("=" * 50)

if "result" in locals():
    print(
        f"Fixed n,m:     Reduced χ² = {result.reduced_chi_squared:.4f}, R² = {result.r_squared:.4f}"
    )

if "result_free" in locals():
    print(
        f"Free n,m:      Reduced χ² = {result_free.reduced_chi_squared:.4f}, R² = {result_free.r_squared:.4f}"
    )

if "result_grid" in locals():
    print(
        f"Grid search:   Reduced χ² = {result_grid.reduced_chi_squared:.4f}, R² = {result_grid.r_squared:.4f}"
    )

print("\nExpected behavior:")
print("- Reduced χ² should be close to 1.0 for well-calibrated uncertainties")
print("- R² should be close to 1.0 for good fit")
print("- Fitted parameters should be close to true values within uncertainties")